In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1)

In [3]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

In [4]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [5]:
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


In [6]:
W = torch.zeros((2,1),requires_grad =True)
b = torch.zeros(1,requires_grad=True)

# Logistic Regression 의 cost 구하기

In [7]:
h = 1/(1+torch.exp(-(x_train.matmul(W)+b)))
h= torch.sigmoid((x_train.matmul(W)+b))

In [8]:
print(h)
print(h.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)
torch.Size([6, 1])


In [9]:
losses = -(y_train * torch.log(h) +(1-y_train)*(torch.log(1-h)))
print(losses)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward0>)


In [10]:
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


In [11]:
F.binary_cross_entropy(h,y_train) # logistic regression의 cost를 구할 수있음

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)

# Logistic Regression

In [12]:
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=1)
print(W)
nb_epochs = 1000
for epoch in range(nb_epochs + 1):


    h = torch.sigmoid(x_train.matmul(W) + b)
    cost = F.binary_cross_entropy(h,y_train) 

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))
print(W)

tensor([[0.],
        [0.]], requires_grad=True)
Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031672
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852
tensor([[3.2530],
        [1.5179]], requires_grad=True)


# Evaluation

In [13]:
h = torch.sigmoid(x_train.matmul(W)+b)
print(h)

tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01],
        [9.9969e-01]], grad_fn=<SigmoidBackward0>)


In [16]:
prediction = h >=torch.FloatTensor([0.5])
print(prediction.int())

tensor([[0],
        [0],
        [0],
        [1],
        [1],
        [1]], dtype=torch.int32)


In [15]:
accuracy = prediction.float() == y_train
print(list(accuracy).count(True)/len(accuracy)*100 ,'%')

100.0 %


# nn.Module 사용한 Higher Implementation

In [49]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2,1)
        self.sigmoid = nn.Sigmoid()
    def forward(self,x):
        return self.sigmoid(self.linear(x))

In [50]:
model =  BinaryClassifier()

In [51]:

optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):

    hypothesis = model(x_train)

    cost = F.binary_cross_entropy(hypothesis, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))

Epoch    0/100 Cost: 0.595946 Accuracy 66.67%
Epoch   10/100 Cost: 0.707534 Accuracy 83.33%
Epoch   20/100 Cost: 0.638608 Accuracy 83.33%
Epoch   30/100 Cost: 0.540139 Accuracy 83.33%
Epoch   40/100 Cost: 0.452255 Accuracy 83.33%
Epoch   50/100 Cost: 0.368146 Accuracy 83.33%
Epoch   60/100 Cost: 0.288984 Accuracy 83.33%
Epoch   70/100 Cost: 0.220561 Accuracy 83.33%
Epoch   80/100 Cost: 0.173926 Accuracy 100.00%
Epoch   90/100 Cost: 0.151975 Accuracy 100.00%
Epoch  100/100 Cost: 0.140495 Accuracy 100.00%
